In [1]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [2]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio

# Notes:
Consider every hexagon as an entity, and sensor data as properties

In [3]:
import pandas as pd
import geopandas as gpd

from geojson.geometry import Point, Polygon, MultiLineString


In [4]:
gdf_1 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_aggr_weather_irradiation.pkl')
gdf_1

,hexagon_id,hexagon_x,hexagon_y,datetime,year,month,day,hour,minute,weekday,altitude,data,interpolated,geometry,latitude,longitude
0,36,3,3,2022-01-01 00:00:00,2022,1,1,0,0,5,646.0,1.000000,0.0,"POLYGON ((-418731.513 4927969.963, -420105.448...",40.412544,-3.758228
1,19,-3,0,2022-01-01 00:00:00,2022,1,1,0,0,5,629.0,3.000000,0.0,"POLYGON ((-403073.982 4922121.466, -404447.305...",40.372541,-3.617604
2,33,-2,-3,2022-01-01 00:00:00,2022,1,1,0,0,5,660.0,1.000000,0.0,"POLYGON ((-402147.668 4934084.980, -403522.534...",40.454353,-3.609281
3,33,-2,-3,2022-01-01 00:15:00,2022,1,1,0,15,5,660.0,1.000000,0.0,"POLYGON ((-402147.668 4934084.980, -403522.534...",40.454353,-3.609281
4,19,-3,0,2022-01-01 00:15:00,2022,1,1,0,15,5,629.0,3.000000,0.0,"POLYGON ((-403073.982 4922121.466, -404447.305...",40.372541,-3.617604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188635,90,4,5,2022-12-31 22:45:00,2022,12,31,22,45,5,646.0,1.083828,1.0,"POLYGON ((-422852.800 4924228.778, -424226.216...",40.386954,-3.795243
3188636,90,4,5,2022-12-31 23:00:00,2022,12,31,23,0,5,646.0,1.083828,1.0,"POLYGON ((-422852.800 4924228.778, -424226.216...",40.386954,-3.795243
3188637,90,4,5,2022-12-31 23:15:00,2022,12,31,23,15,5,646.0,1.083828,1.0,"POLYGON ((-422852.800 4924228.778, -424226.216...",40.386954,-3.795243
3188638,90,4,5,2022-12-31 23:30:00,2022,12,31,23,30,5,646.0,1.083828,1.0,"POLYGON ((-422852.800 4924228.778, -424226.216...",40.386954,-3.795243


In [5]:
gdf_2 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_aggr_weather_P.pkl')
gdf_3 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_aggr_weather_rain.pkl')
gdf_4 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_aggr_weather_RH.pkl')
gdf_5 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_aggr_weather_temperature.pkl')
gdf_6 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_aggr_weather_wind_dir.pkl')
gdf_7 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_aggr_weather_wind_speed.pkl')



In [6]:


gdf_list= [gdf_1, gdf_2, gdf_3, gdf_4, gdf_5, gdf_6, gdf_7 ]
gdf_property_list=["Irradiation", "P", "Rain", "RH", "Temperature", "Wind_Direction", "Wind_Speed"]
entity_id_prefix =  "urn:ngsi-ld:WeatherObserved:Madrid"
entity_type = 'WeatherObserved'


1- Check context and add context (smart data model link) to the @context tag

2- Geometry: Change geometry to location. Location is a geoproperty. Give the point values as "coordinates". Keep the observedat on every property.

3- Convert coordinates to lon/lat

4- Add no2, so2 as property (not as a separate entity). One entity per hexagon
. 
5- Add district instead of hexagons


In [ ]:

example_flag=False
for property_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True
    prop =gdf_property_list[property_index]

    hexagon_id_list = tmp_gdf['hexagon_id'].unique()
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]
        
        entity_obj = Entity(entity_type,entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
        print("Info: Entity ID: "  + entity_id_prefix + ':Hexagon:' + str(hexagon_id))
        counter =0
        for index, row in hexagon_gdf.iterrows():
            lat=0
            lon=0
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type, entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if feature_list[f] =='type': feature_list[f] ='road_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
                if feature_list[f] == 'lat': lat = row[f]
                if feature_list[f] == 'lon': lon=row[f]
                if lat !=0 and lon !=0: # both values are obtained for location, add the location attribute 
                    location_value = Point((lat, lon))
                    entity_obj.add_geoprop("location", location_value, observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S')) # MAKE THIS A POINT (SIVA EXAMPLE)
                elif feature_list[f] == 'data':
                    entity_obj.add_prop(prop, str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
                elif(feature_list[f] not in ['datetime','type']):
                    entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
        print("... published data for hexagon: " +  str(hexagon_id))

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully' 
        
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


Info: Unique hexagons for the dataset : [36 19 33  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21
 22 23 24 25 26 27 28 29 30 31 32 34 35 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90]
Info: Entity ID: urn:ngsi-ld:WeatherObserved:Madrid:Hexagon:36
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:WeatherObserved:Madrid:Hexagon:36', 'type': 'WeatherObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'hexagon_id': {'type': 'Property', 'value': '36', 'observedAt': '2022-01-01T00:00:00Z'}, 'hexagon_x': {'type': 'Property', 'value': '3', 'observedAt': '2022-01-01T00:00:00Z'}, 'hexagon_y': {'type': 'Property', 'value': '3', 'observedAt': '2022-01-01T00:00:00Z'}, 'year': {'type': 'Property', 'value': '2022', 'observedAt': '2022-01-01T00:00:00Z'}, 'month': {'type': 'Prope

# Query temporal entity from Scorpio and get it as Pandas

In [ ]:

client_obj = Client("localhost", "9090")
pandas_data= client_obj.get_temporal("urn:ngsi-ld:AirQualityObserved:Madrid:Hexagon:0", entityType="AirQualityObserved", pandasSeries=True, attribute="value")
print(pandas_data)
print(type(pandas_data))



## BACKUP CODE

import time


for entity_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
        
    feature_list= tmp_gdf.columns.tolist()
    for index, row in tmp_gdf.iterrows():
        if index %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
            entity_obj = Entity(entity_type_list[entity_index], entity_id_list[entity_index]) # (re-)initialize the entity
        for i in range(len(feature_list)):
            if feature_list[i] =='type': feature_list[i] ='road_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
            if(feature_list[i] not in ['datetime','type']):
                entity_obj.add_prop(feature_list[i], str(row[i]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
        if index == 0:
            print('INFO: Initial row of the dataset to be published - as an example on how data looks.')
            print(entity_obj)
            
        #ingestion_starttime= time.perf_counter()
        temp_objects.append(entity_obj)
        if (index %num_rows_batch_publish==0 or index == len(tmp_gdf)-1) and  index!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
            publish(entity_obj.to_dict()) # publish the entity
    
        f(index%10000 ==0):
            print("... published data count: " +  str(index))

        #print('Info: Ingestion took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds for ' +  str(num_rows_batch_publish) + ' objects')

    print('Info: Dataset is published successfully with entity ID: ' + str(entity_id_list[entity_index]))
